# Deep learning-based synthetic CT generation for MR-guided radiotherapy optimization

---



Important: A GPU runtime should be selected

In [ ]:
import os
import sys
import shutil
import matplotlib.pyplot as plt
import cv2
from google.colab import drive

Mount to Google Drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Dowload our repository

In [ ]:
os.chdir('/content')
try:
  shutil.rmtree('/content/DL-Project-2020')
except:
  pass

!git clone https://github.com/deltahue/DL-Project-2020

os.chdir("DL-Project-2020/contrastive-unpaired-translation-master") 

Cloning into 'DL-Project-2020'...
remote: Enumerating objects: 321, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 1624 (delta 157), reused 252 (delta 114), pack-reused 1303
Receiving objects: 100% (1624/1624), 16.45 MiB | 8.69 MiB/s, done.
Resolving deltas: 100% (824/824), done.


Install modules


In [ ]:
!pip install -r requirements.txt
!pip install torchio
!pip install pytorch-lightning
!pip install pytorch-fid
!pip install patchify

Unpack the dataset. Together with the report, we have submitted a train.rar file and a test.rar file. To make the dataset accessible to Google Colab, a 'dataset' folder has to be placed in 'MyDrive'  in Google Drive where the train.rar and test.rar are present.

In [ ]:
!unrar x "/content/drive/MyDrive/dataset/train.rar" "/content/dataset/train/"
!unrar x "/content/drive/MyDrive/dataset/test.rar" "/content/dataset/test/"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/MyDrive/dataset/train.rar

Creating    /content/dataset/train                                    OK
Creating    /content/dataset/train/train                              OK
Creating    /content/dataset/train/train/trainA                       OK
Extracting  /content/dataset/train/train/trainA/PAT8-MRSim-1.nii           0%  OK 
Extracting  /content/dataset/train/train/trainA/PAT8-MRSim-10.nii          0%  OK 
Extracting  /content/dataset/train/train/trainA/PAT8-MRSim-100.nii         0%  OK 
Extracting  /content/dataset/train/train/trainA/PAT8-MRSim-101.nii         0%  OK 
Extracting  /content/dataset/train/train/trainA/PAT8-MRSim-102.nii         0%  OK 
Extracting  /content/dataset/train/train/trainA/PAT8-MRSim-103.nii         1%  OK 
Extracting  /content/dataset/train/train/trainA/PAT8-MRSim-104.nii         1%  OK 
Extracting

Start the training

In [ ]:
# CUT
!python -u train.py --dataroot '/content/dataset/train/train' --name experiment_CUT --batch_size 1 --max_dataset_size 50  --output_nc 1 --input_nc 1 --verbose --lr 0.0001 --preprocess none --crop_size 256 --model cut

# FastCUT
!python -u train.py --dataroot '/content/dataset/train/train' --name experiment_FastCUT  --batch_size 1 --max_dataset_size 50  --output_nc 1 --input_nc 1 --verbose --lr 0.0001 --preprocess none --crop_size 256 --model cut --mode FastCUT

# CycleGAN
!python -u train.py --dataroot '/content/dataset/train/train' --name experiment_CycleGAN --batch_size 1 --max_dataset_size 50  --output_nc 1 --input_nc 1 --verbose --lr 0.0001 --preprocess none --crop_size 256 --model cycle_gan


# Evaluation


Load the weights (ignore if the current runtime was also used for training). The following cell copies the dowloaded weights of each model into the repository folder so that they can be used for testing. Make sure to have stored the network weights in the corresponding folders in your MyDrive folder: 
- CUT weights: Trained_CUT_Crop256
- FastCUT weights: Trained_FastCUT_Crop256
- CycleGAN weights: Trained_CycleGAN_Crop256

In [ ]:
os.makedirs(os.path.dirname('/content/DL-Project-2020/contrastive-unpaired-translation-master/checkpoints/experiment_CUT/'), exist_ok=True)
os.makedirs(os.path.dirname('/content/DL-Project-2020/contrastive-unpaired-translation-master/checkpoints/experiment_FastCUT/'), exist_ok=True)
os.makedirs(os.path.dirname('/content/DL-Project-2020/contrastive-unpaired-translation-master/checkpoints/experiment_CycleGAN/'), exist_ok=True)

# CUT
shutil.copyfile('/content/drive/MyDrive/Trained_CUT_Crop256/latest_net_G.pth', '/content/DL-Project-2020/contrastive-unpaired-translation-master/checkpoints/experiment_CUT/latest_net_G.pth')

# FastCUT
shutil.copyfile('/content/drive/MyDrive/Trained_FastCUT_Crop256/latest_net_G.pth', '/content/DL-Project-2020/contrastive-unpaired-translation-master/checkpoints/experiment_FastCUT/latest_net_G.pth')

# CycleGAN
shutil.copyfile('/content/drive/MyDrive/Trained_CycleGAN_Crop256/latest_net_G_A.pth', '/content/DL-Project-2020/contrastive-unpaired-translation-master/checkpoints/experiment_CycleGAN/latest_net_G_A.pth')
shutil.copyfile('/content/drive/MyDrive/Trained_CycleGAN_Crop256/latest_net_G_B.pth', '/content/DL-Project-2020/contrastive-unpaired-translation-master/checkpoints/experiment_CycleGAN/latest_net_G_B.pth')

# shutil.copytree('/content/drive/MyDrive/cycleGAN/', '/content/DL-Project-2020/contrastive-unpaired-translation-master/checkpoints/CycleGAN')


'/content/DL-Project-2020/contrastive-unpaired-translation-master/checkpoints/experiment_CycleGAN/latest_net_G_B.pth'

Test the models

In [ ]:
  # Test CUT
  # !python test.py --dataroot "/content/dataset/test/test" --name experiment_CUT --crop_size 512 --load_size 512 --num_threads 1 --no_flip  --output_nc 1 --input_nc 1  --model cut
  
  # Test FastCUT
  !python test.py --dataroot "/content/dataset/test/test" --name experiment_FastCUT --crop_size 512 --load_size 512 --num_threads 1 --no_flip  --output_nc 1 --input_nc 1  --model cut --CUT_mode FastCUT
  
  # Test CycleGAN
  # !python test.py --dataroot "/content/dataset/test/test"--name experiment_CycleGAN --crop_size 512 --load_size 512 --num_threads 1 --no_flip  --output_nc 1 --input_nc 1  --model cycle_gan

----------------- Options ---------------
                 CUT_mode: FastCUT                       	[default: CUT]
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 512                           	[default: 256]
                 dataroot: /content/dataset/test/test    	[default: placeholder]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
               easy_label: experiment_name               
                    epoch: latest                        
                     eval: False                         
        flip_equivariance: True                          
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: xavier                        
                 input_nc: 1       

After testing is completed, the results will be shown under the results folder in the repository directory.

